
# Taller de Ciencia de Datos: ETL + EDA Estadístico  
## Caso: Ecuador, Colombia y Perú (PIB, Población, Esperanza de Vida)

**Objetivo del Notebook:**  
Guiar paso a paso el proceso completo de:

1. **ETL (Extract – Transform – Load)** de un conjunto de datos simplificado para Ecuador, Colombia y Perú.
2. **EDA (Exploratory Data Analysis) con enfoque estadístico**, usando medidas de tendencia, dispersión, correlaciones y visualizaciones.

Este Notebook está diseñado para estudiantes que **ya manejan Python básico** y están empezando en Ciencia de Datos.



## 📚 Contenidos

1. **Configuración inicial** (importación de librerías)
2. **Construcción / Extracción de datos crudos (simulación de repositorio)**
3. **ETL**
   - Extract: obtención de los datos
   - Transform: limpieza, normalización y creación de nuevas variables
   - Load: almacenamiento en un CSV limpio
4. **EDA Descriptivo**
   - Dimensiones, tipos de datos
   - Medidas de tendencia central y dispersión
   - Histogramas y boxplots
5. **EDA Estadístico**
   - Agrupaciones (por país y año)
   - Correlaciones y matriz de correlación
   - Análisis de series de tiempo
   - Gráficos de dispersión entre variables (PIB, población, esperanza de vida)
6. **EDA Inferencial (Introductorio)**
   - Prueba de normalidad (Shapiro-Wilk)
   - Correlación de Pearson
7. **Ejercicios propuestos para el estudiante**



## 1️⃣ Configuración inicial

En esta sección importamos las librerías necesarias.

- `pandas`: manejo de datos tabulares.
- `numpy`: operaciones numéricas.
- `matplotlib` y `seaborn`: visualización.
- `scipy.stats`: algunas pruebas estadísticas básicas.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import shapiro, pearsonr

# Configuración estética de gráficos
plt.style.use("default")
sns.set()



## 2️⃣ Construcción / Extracción de datos crudos

En un caso real, estos datos vendrían de un **repositorio de datos** (por ejemplo, Banco Mundial, Kaggle, datahub.io, etc.).  
Para hacer el taller **reproducible sin conexión**, simularemos que ya descargamos los datos y los construiremos directamente en el Notebook.

Las variables serán:

- `Pais`: Ecuador, Colombia, Perú  
- `Anio`: de 2010 a 2020  
- `PIB_per_capita`: PIB per cápita aproximado (USD corrientes, valores ficticios pero verosímiles)  
- `Poblacion`: población total aproximada  
- `Esperanza_vida`: años de esperanza de vida al nacer (ficticios pero razonables)


In [ ]:

# Creamos listas de años y países
anios = list(range(2010, 2021))
paises = ["Ecuador", "Colombia", "Peru"]

# Generamos datos ficticios pero verosímiles
data = []

np.random.seed(42)  # Para reproducibilidad

for pais in paises:
    # Valores base aproximados para cada país
    if pais == "Ecuador":
        pib_base = 4000
        pob_base = 14500000
        vida_base = 74
    elif pais == "Colombia":
        pib_base = 5500
        pob_base = 46000000
        vida_base = 73
    else:  # Peru
        pib_base = 5000
        pob_base = 29000000
        vida_base = 72

    pib = pib_base
    pob = pob_base
    vida = vida_base

    for anio in anios:
        # Simulamos variaciones suaves año a año
        pib = pib * (1 + np.random.normal(0.03, 0.01))    # crecimiento ~3% con ruido
        pob = pob * (1 + np.random.normal(0.012, 0.003))  # crecimiento ~1.2%
        vida = vida + np.random.normal(0.05, 0.03)        # aumento lento

        data.append({
            "Pais": pais,
            "Anio": anio,
            "PIB_per_capita": round(pib, 2),
            "Poblacion": int(pob),
            "Esperanza_vida": round(vida, 2)
        })

# Creamos el DataFrame crudo
df_raw = pd.DataFrame(data)
df_raw.head()



## 3️⃣ Proceso ETL (Extract – Transform – Load)

En este caso:

- **Extract:** ya lo realizamos simulando los datos (en la vida real sería leer un CSV, una API, una base de datos, etc.).  
- **Transform:** vamos a limpiar, verificar tipos, generar nuevas variables.  
- **Load:** guardaremos un archivo `datos_paises_limpio.csv` con los datos transformados.



### 3.1 Transform: inspección y limpieza


In [ ]:

# Copiamos el DataFrame crudo para no modificar el original
df = df_raw.copy()

# Vistazo general
print("Dimensiones:", df.shape)
print("\nInformación general:")
print(df.info())

print("\nValores nulos por columna:")
print(df.isnull().sum())

df.head()



En este punto podríamos:

- Eliminar filas con datos nulos.
- Corregir tipos de datos (si fuera necesario).
- Renombrar columnas para darles un nombre más amigable.


In [ ]:

# Aseguramos tipos de datos correctos
df["Anio"] = df["Anio"].astype(int)
df["PIB_per_capita"] = df["PIB_per_capita"].astype(float)
df["Poblacion"] = df["Poblacion"].astype(int)
df["Esperanza_vida"] = df["Esperanza_vida"].astype(float)

# Creamos columna de año en formato datetime (opcional para series de tiempo)
df["Fecha"] = pd.to_datetime(df["Anio"], format="%Y")

df.head()



### 3.2 Creación de nuevas variables

Creamos, por ejemplo:

- `PIB_total` = PIB per cápita × población  
- `Crecimiento_PIB_per_capita` = % de cambio anual del PIB per cápita por país  
- `Crecimiento_Poblacion` = % de cambio anual de la población por país


In [ ]:

# PIB total (millones de USD aproximadamente)
df["PIB_total"] = df["PIB_per_capita"] * df["Poblacion"]

# Ordenamos por país y año antes de calcular tasas
df = df.sort_values(by=["Pais", "Anio"])

# Tasas de crecimiento porcentual
df["Crecimiento_PIB_per_capita"] = df.groupby("Pais")["PIB_per_capita"].pct_change() * 100
df["Crecimiento_Poblacion"] = df.groupby("Pais")["Poblacion"].pct_change() * 100

df.head(10)



### 3.3 Load: guardado de datos limpios

Guardamos el DataFrame resultante en un archivo CSV para usos posteriores.


In [ ]:

df.to_csv("datos_paises_limpio.csv", index=False)
print("✅ Archivo 'datos_paises_limpio.csv' guardado correctamente.")



## 4️⃣ EDA Descriptivo

En esta sección haremos un análisis exploratorio básico:

- Dimensiones y estructura
- Estadísticos descriptivos (media, mediana, desviación estándar)
- Distribuciones (histogramas)
- Boxplots para observar dispersión y outliers


In [ ]:

# Cargamos el archivo limpio (simulando que venimos "desde cero")
df_eda = pd.read_csv("datos_paises_limpio.csv")

print("Dimensiones:", df_eda.shape)
df_eda.head()


In [ ]:

# Estadísticos descriptivos generales
df_eda[["PIB_per_capita", "Poblacion", "Esperanza_vida", "PIB_total"]].describe()



### 4.1 Histogramas: distribución de variables numéricas


In [ ]:

fig, axes = plt.subplots(1, 3, figsize=(18, 4))

sns.histplot(df_eda["PIB_per_capita"], kde=True, ax=axes[0])
axes[0].set_title("Distribución PIB per cápita")

sns.histplot(df_eda["Poblacion"], kde=True, ax=axes[1])
axes[1].set_title("Distribución de Población")

sns.histplot(df_eda["Esperanza_vida"], kde=True, ax=axes[2])
axes[2].set_title("Distribución de Esperanza de vida")

plt.tight_layout()
plt.show()



### 4.2 Boxplots: detección visual de outliers


In [ ]:

fig, axes = plt.subplots(1, 3, figsize=(18, 4))

sns.boxplot(x=df_eda["PIB_per_capita"], ax=axes[0])
axes[0].set_title("Boxplot PIB per cápita")

sns.boxplot(x=df_eda["Poblacion"], ax=axes[1])
axes[1].set_title("Boxplot Población")

sns.boxplot(x=df_eda["Esperanza_vida"], ax=axes[2])
axes[2].set_title("Boxplot Esperanza de vida")

plt.tight_layout()
plt.show()



## 5️⃣ EDA Estadístico

Ahora profundizamos en el análisis:

- Estadísticos por país
- Comparaciones entre países
- Matriz de correlación
- Análisis de series de tiempo (evolución por año)
- Relación entre desarrollo (PIB, esperanza de vida) y población



### 5.1 Estadísticos por país


In [ ]:

# Media de cada variable por país
estadisticos_pais = df_eda.groupby("Pais")[["PIB_per_capita", "Poblacion", "Esperanza_vida"]].agg(
    ["mean", "std", "min", "max"]
)
estadisticos_pais



### 5.2 Comparación gráfica de PIB per cápita promedio por país


In [ ]:

pib_promedio = df_eda.groupby("Pais")["PIB_per_capita"].mean().sort_values(ascending=False)

plt.figure(figsize=(6,4))
sns.barplot(x=pib_promedio.index, y=pib_promedio.values)
plt.title("PIB per cápita promedio (2010-2020)")
plt.ylabel("PIB per cápita (USD)")
plt.xlabel("País")
plt.show()



### 5.3 Matriz de correlación

Analizamos la correlación entre:

- PIB per cápita  
- Población  
- Esperanza de vida  
- PIB total


In [ ]:

corr = df_eda[["PIB_per_capita", "Poblacion", "Esperanza_vida", "PIB_total"]].corr()
corr


In [ ]:

plt.figure(figsize=(6,4))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Matriz de correlación")
plt.show()



### 5.4 Series de tiempo: evolución del PIB per cápita

Graficamos cómo ha cambiado el PIB per cápita en el tiempo para cada país.


In [ ]:

plt.figure(figsize=(10,5))
sns.lineplot(data=df_eda, x="Anio", y="PIB_per_capita", hue="Pais", marker="o")
plt.title("Evolución del PIB per cápita (2010-2020)")
plt.ylabel("PIB per cápita (USD)")
plt.show()



### 5.5 Relación entre PIB per cápita y esperanza de vida


In [ ]:

plt.figure(figsize=(7,5))
sns.scatterplot(data=df_eda, x="PIB_per_capita", y="Esperanza_vida", hue="Pais")
plt.title("PIB per cápita vs Esperanza de vida")
plt.xlabel("PIB per cápita (USD)")
plt.ylabel("Esperanza de vida (años)")
plt.show()



## 6️⃣ EDA Inferencial (Introductorio)

En esta sección introducimos algunas ideas básicas de estadística inferencial:

- **Prueba de normalidad (Shapiro-Wilk)** sobre el PIB per cápita.
- **Correlación de Pearson** entre PIB per cápita y esperanza de vida.



### 6.1 Prueba de normalidad (Shapiro-Wilk) para PIB per cápita

> **Hipótesis nula (H0):** los datos provienen de una distribución normal.  
> Si el p-valor < 0.05, rechazamos H0 (no son normales).


In [ ]:

# Tomamos una muestra (por si el N fuera muy grande)
muestra_pib = df_eda["PIB_per_capita"].sample(min(100, len(df_eda)), random_state=42)

stat, p_value = shapiro(muestra_pib)
print(f"Estadístico Shapiro-Wilk: {stat:.4f}")
print(f"p-valor: {p_value:.4f}")



### 6.2 Correlación de Pearson entre PIB per cápita y Esperanza de vida

> **Hipótesis nula:** no hay correlación lineal entre las variables.  
> Si el p-valor < 0.05, hay evidencia de correlación lineal significativa.


In [ ]:

corr_pearson, p_val = pearsonr(df_eda["PIB_per_capita"], df_eda["Esperanza_vida"])
print(f"Correlación de Pearson: {corr_pearson:.4f}")
print(f"p-valor: {p_val:.4f}")



## 7️⃣ Ejercicios propuestos para el estudiante

1. **Medidas descriptivas por país**
   - Calcula la media, mediana y desviación estándar de la esperanza de vida para cada país.
   - Interpreta: ¿Qué país muestra mayor estabilidad (menor desviación)?

2. **Análisis de crecimiento**
   - Utiliza las columnas `Crecimiento_PIB_per_capita` y `Crecimiento_Poblacion`.
   - Calcula el crecimiento promedio del PIB per cápita entre 2010 y 2020 por país.
   - ¿Qué país ha crecido más rápido según estos datos simulados?

3. **Boxplots por país**
   - Crea un boxplot de `PIB_per_capita` por país (en el eje X el país, en el Y el PIB per cápita).
   - ¿Qué diferencias encuentras en la dispersión?

4. **Profundiza en la correlación**
   - Calcula la correlación entre `Poblacion` y `PIB_total`.
   - Interpreta el resultado: ¿es esperable ese signo y magnitud? ¿por qué?

5. **Extensión opcional (para investigar en casa)**
   - Reemplaza estos datos simulados con un dataset real descargado del **Banco Mundial** o **datahub.io**.
   - Repite el EDA y compara resultados.
